In [1]:
from pyspark.sql import SparkSession, functions as f, types as t

In [2]:
spark = SparkSession.builder.master('local[3]').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/20 19:24:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
print(spark.version)

3.3.0


In [4]:
actor_df = spark.read.csv('./data/actor.csv', header=True, inferSchema=True)
address_df = spark.read.csv('./data/address.csv', header=True, inferSchema=True)
category_df = spark.read.csv('./data/category.csv', header=True, inferSchema=True)
city_df = spark.read.csv('./data/city.csv', header=True, inferSchema=True)
country_df = spark.read.csv('./data/country.csv', header=True, inferSchema=True)
customer_df = spark.read.csv('./data/customer.csv', header=True, inferSchema=True)
film_df = spark.read.csv('./data/film.csv', header=True, inferSchema=True)
film_actor_df = spark.read.csv('./data/film_actor.csv', header=True, inferSchema=True)
film_category_df = spark.read.csv('./data/film_category.csv', header=True, inferSchema=True)
inventory_df = spark.read.csv('./data/inventory.csv', header=True, inferSchema=True)
language_df = spark.read.csv('./data/language.csv', header=True, inferSchema=True)
payment_df = spark.read.csv('./data/payment.csv', header=True, inferSchema=True)
rental_df = spark.read.csv('./data/rental.csv', header=True, inferSchema=True)
staff_df = spark.read.csv('./data/staff.csv', header=True, inferSchema=True)
store_df = spark.read.csv('./data/store.csv', header=True, inferSchema=True)

# Домашнє завдання на тему Spark SQL

Задачі з домашнього завдання на SQL потрібно розвʼязати за допомогою Spark SQL DataFrame API.

- Дампи таблиць знаходяться в папці `data`. Датафрейми таблиць вже створені в клітинці вище.
- Можете створювати стільки нових клітинок, скільки вам необхідно.
- Розвʼязок кожної задачі має бути відображений в самому файлі (використати метод `.show()`)
- код має бути оформлений у відповідності із одним із стилем, показаним лектором на занятті 13.

**Увага!**
Використовувати мову запитів SQL безпосередньо забороняється, потрібно використовувати виключно DataFrame API!


1.
Вивести кількість фільмів в кожній категорії.
Результат відсортувати за спаданням.

In [34]:
category_df.join(
    film_category_df,
    "category_id",
    "inner"
).join(
    film_df,
    "film_id",
    "inner"
).select(
    "name"
).groupBy(
    "name"
).agg(
    f.count("name").alias('count_films')
).orderBy(
    f.desc("count_films")
).show()

+-----------+-----------+
|       name|count_films|
+-----------+-----------+
|     Sports|         74|
|    Foreign|         73|
|     Family|         69|
|Documentary|         68|
|  Animation|         66|
|     Action|         64|
|        New|         63|
|      Drama|         62|
|      Games|         61|
|     Sci-Fi|         61|
|   Children|         60|
|     Comedy|         58|
|     Travel|         57|
|   Classics|         57|
|     Horror|         56|
|      Music|         51|
+-----------+-----------+



2.
Вивести 10 акторів, чиї фільми брали на прокат найбільше.
Результат відсортувати за спаданням.

In [50]:
actor_df.join(
    film_actor_df,
    "actor_id",
    "inner"
).join(
    inventory_df.join(
        rental_df,
        "inventory_id",
        "inner"
    ).select(
        "film_id"
    ).groupBy(
        "film_id"
    ).agg(
        f.count("film_id").alias("films_count")
    ),
    "film_id",
    "inner"
).select(
    f.concat(
        'last_name', f.lit(' ') ,'first_name'
    ).alias("actors"),
    "films_count"
).orderBy(
    f.desc("films_count")
).limit(
    10
).show()


+-----------------+-----------+
|           actors|films_count|
+-----------------+-----------+
|      HACKMAN TIM|         34|
|     PHOENIX GARY|         34|
|      TEMPLE BURT|         34|
|   AKROYD KIRSTEN|         34|
|   DENCH CHARLIZE|         34|
|     CRAWFORD RIP|         34|
|      MIRANDA TOM|         33|
|      ALLEN MERYL|         33|
|        DEAN JUDY|         33|
|SILVERSTONE JAYNE|         33|
+-----------------+-----------+



3.
Вивести категорія фільмів, на яку було витрачено найбільше грошей
в прокаті

In [210]:
category_df.join(
    film_category_df,
    "category_id",
    "inner"
).join(
    inventory_df,
    "film_id",
    "inner"
).join(
    rental_df,
    "inventory_id",
    "inner"
).join(
    payment_df,
    "rental_id",
    "inner"
).select(
    f.col("name").alias("category_name"),
    "amount"
).groupBy(
        "category_name"
).agg(
    f.sum(
        f.round("amount")
    ).alias("category_sum_amount")
).orderBy(
    f.desc("category_sum_amount")
).limit(
    1
).select(
    "category_name"
).show()

+-------------+
|category_name|
+-------------+
|       Sports|
+-------------+



4.
Вивести назви фільмів, яких не має в inventory.

In [142]:
film_df.join(
    inventory_df,
    "film_id",
    "left"
).select(
    "title"
).where(
    f.col("inventory_id").isNull() == True
).show()

+--------------------+
|               title|
+--------------------+
|      ALICE FANTASIA|
|         APOLLO TEEN|
|      ARGONAUTS TOWN|
|       ARK RIDGEMONT|
|ARSENIC INDEPENDENCE|
|   BOONDOCK BALLROOM|
|       BUTCH PANTHER|
|       CATCH AMISTAD|
| CHINATOWN GLADIATOR|
|      CHOCOLATE DUCK|
|COMMANDMENTS EXPRESS|
|    CROSSING DIVORCE|
|     CROWDS TELEMARK|
|    CRYSTAL BREAKING|
|          DAZED PUNK|
|DELIVERANCE MULHO...|
|   FIREHOUSE VIETNAM|
|       FLOATS GARDEN|
|FRANKENSTEIN STRA...|
|  GLADIATOR WESTWARD|
+--------------------+
only showing top 20 rows



5.
Вивести топ 3 актори, які найбільше зʼявлялись в категорії фільмів “Children”

In [170]:
actor_df.join(
    film_actor_df,
    "actor_id",
    "inner"
).join(
    film_category_df,
    "film_id",
    "inner"
).join(
    category_df,
    "category_id",
    "inner"
).join(
    film_df,
    "film_id",
    "inner"
).select(
    f.concat(
        'last_name', f.lit(' ') ,'first_name'
    ).alias("actors"),
    "film_id",
    "name"
).groupBy(
    f.col("actors"),
    "name"
).agg(
    f.count("film_id").alias("film_count")
).orderBy(
    f.desc("film_count")
).where(
    f.col("name") == 'Children'
).limit(
    3
).select(
    "actors"
).show()

+------------+
|      actors|
+------------+
|VOIGHT HELEN|
| DAVIS SUSAN|
|  CRUZ RALPH|
+------------+



Stop Spark session:

In [211]:
spark.stop()